# EEG GAN

In [ ]:
import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

In [ ]:
def single_channel_plot(data, channel, title):
    plt.figure(figsize=(20, 3))
    plt.plot(data[channel, :])
    plt.title(title)
    plt.show()

def plot_all_channels(data, title):
    plt.figure(figsize=(20, 10))
    plt.plot(data.T)
    plt.title(title)
    plt.show()

def plot_all_channels_subplots(data):
    plt.figure(figsize=(20, 10))
    for i in range(64):
        plt.subplot(8, 8, i+1)
        plt.plot(data[i, :])
        plt.title(f"Channel {i}")
    plt.show()

main_folder = './EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0'
subdirectories = [f.path for f in os.scandir(main_folder) if f.is_dir()]
opened_files = []
closed_files = []

for subdirectory in subdirectories:
    files = os.listdir(subdirectory)
    if len(files) > 0:
        for file in files:
            if file[-6:] == '01.edf':
                # This is data for eyes opened
                eyes_opened = os.path.join(subdirectory, file)
                opened_files.append(eyes_opened)
            if file[-6:] == '02.edf':
                # This is data for eyes closed
                eyes_closed = os.path.join(subdirectory, file)
                closed_files.append(eyes_closed)
    else:
        print(f"No files found in {subdirectory}")

large_open_data_raw = []
large_closed_data_raw = []

flag = True

for i in range(len(opened_files)):
    if i not in [1,4,5,6,12,13,15,17,19,22,29,31,33,34.47,50,51,58,60,71,76,77,79,81,86,87,88,90,95,100]:
        data1 = mne.io.read_raw_edf(opened_files[i], preload=True, verbose=False).get_data(verbose=False)
        data2 = mne.io.read_raw_edf(closed_files[i], preload=True, verbose=False).get_data(verbose=False)
        for j in range(max(len(data1), len(data2))):
            if len(data1[j]) != 9760 or len(data2[j]) != 9760:
                flag = False
        if flag:
            large_open_data_raw.append(data1)
            large_closed_data_raw.append(data2)
        flag = True

print("Length of open dataset:", large_open_data_raw.__len__())
print("Length of closed dataset:", large_closed_data_raw.__len__())

for i in range(len(large_open_data_raw)):
    if len(large_open_data_raw[i]) != 64:
        print(i)
    for j in range(len(large_open_data_raw[i])):
        if len(large_open_data_raw[i][j]) != 9760:
            print(i)

for i in range(len(large_closed_data_raw)):
    if len(large_closed_data_raw[i]) != 64:
        print(i)
    for j in range(len(large_closed_data_raw[i])):
        if len(large_closed_data_raw[i][j]) != 9760:
            print(i)

In [ ]:
# Removing unwannted variables
del opened_files
del closed_files
del subdirectories

In [ ]:
import wandb
import random

wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

## 